# Example of Ask/Tell interface with DEAP 

In [1]:
import random
import array
import numpy

from deap import base, benchmarks, creator, tools, algorithms
from deap import cma

In [2]:
import numpy as np
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod

### Input

In [3]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [4]:
# Create target current traces
output_traces = 10*nS*input_traces

In [5]:
input = input_traces
output = output_traces

In [6]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])

### Setup The Model for Optimization

In [7]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

In [8]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [9]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.00s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [10]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [11]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [12]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [13]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

#### Record error  
additional differential equation calculating the error

In [14]:
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

# store the state of the network
store()

In [15]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
            
    return d

In [16]:
def calc_error(params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    
    return array(e)

In [17]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Ask and Tel with calc_error Function

In [18]:
start_scope()

In [19]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [20]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [21]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [22]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### set up DEAP

In [91]:
NDIM = 2
IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

strategy = cma.Strategy(centroid=[2.0]*NDIM, sigma=1.0)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -2, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

toolbox.register("generate", strategy.generate, creator.__dict__["Individual"])
toolbox.register("update", strategy.update)

In [92]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

In [93]:
stats = None
ngen = 1

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

for gen in range(ngen):
    # Generate a new population
    population = toolbox.generate()
    print(population, '\n')

[[0.08107796662053635, 2.5448835038932724], [2.088386511821421, 1.8280859725084158], [1.4073648222973298, 1.3051473206692639], [3.38201206561909, 1.0854392884433053], [1.2965383364806171, 2.339765377892572], [2.798891323449465, 0.8627834162775763]] 



In [94]:
population

[[0.08107796662053635, 2.5448835038932724],
 [2.088386511821421, 1.8280859725084158],
 [1.4073648222973298, 1.3051473206692639],
 [3.38201206561909, 1.0854392884433053],
 [1.2965383364806171, 2.339765377892572],
 [2.798891323449465, 0.8627834162775763]]

In [84]:
fitnesses = calc_error(np.array(population))

for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    ind.fitness.values = (fit,)

In [85]:
hof.update(population)

In [86]:
toolbox.update(population)

In [87]:
record = stats.compile(population) if stats is not None else {}
logbook.record(gen=gen, nevals=len(population), **record)

print(logbook.stream)

gen	nevals
0  	6     


In [88]:
hof[0]

[3.4351946872979995, 0.653748703363412]

In [89]:
hof[0].fitness.values[0]

5.043411354698068

In [90]:
print("Best individual is ", hof[0], hof[0].fitness.values[0])

Best individual is  [3.4351946872979995, 0.653748703363412] 5.043411354698068


### Different Approach

In [96]:
FITCLSNAME = "FIT_TYPE"
INDCLSNAME = "IND_TYPE"
creator.create(FITCLSNAME, base.Fitness, weights=(-1.0,))
creator.create(INDCLSNAME, list, fitness=creator.__dict__[FITCLSNAME])
# # HV_THRESHOLD = 116.0        # 120.777 is Optimal value

In [121]:
NDIM = 2
BOUND_LOW, BOUND_UP = 0.0, 1.0
MU, LAMBDA = 10, 10
NGEN = 5

In [122]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -2, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

In [123]:
# The MO-CMA-ES algorithm takes a full population as argument
population = [creator.__dict__[INDCLSNAME](x) for x in numpy.random.uniform(BOUND_LOW, BOUND_UP, (MU, NDIM))]


In [124]:
strategy = cma.StrategyMultiObjective(population, sigma=1.0, mu=MU, lambda_=LAMBDA)

toolbox.register("generate", strategy.generate, creator.__dict__[INDCLSNAME])
toolbox.register("update", strategy.update)

In [125]:
for gen in range(NGEN):
    # Generate a new population
    population = toolbox.generate()

In [126]:
# Evaluate the individuals
fitnesses = calc_error(np.array(population))

for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    ind.fitness.values = (fit,)

[[-0.82416153 -0.53421864]
 [ 0.77483529  0.90733393]
 [ 1.48933694  1.67685686]
 [ 0.64835172  1.35265723]
 [ 1.34713574 -0.91266888]
 [-0.66968839  0.60213775]
 [ 0.69577141  1.02117625]
 [-0.49974395  0.5941497 ]
 [-1.0966259  -0.44145559]
 [ 0.10444761  1.8731736 ]]
[-0.8241615270924183, -0.5342186370654101] 0.1938486337747691
[0.7748352930129296, 0.9073339302322371] 0.49425729965908716
[1.4893369386880462, 1.6768568608468029] 6.23703105349027
[0.6483517236631182, 1.3526572259143101] 0.7691248674884398
[1.3471357414842524, -0.9126688767248644] 1.5116428666418555
[-0.6696883911965256, 0.6021377513219726] 0.16260625740044873
[0.6957714140768014, 1.0211762531609159] 0.5048177041436573
[-0.4997439457256274, 0.5941497004981493] 0.08816309908598141
[-1.0966258997986085, -0.4414555907783102] 0.23436407461797454
[0.1044476106593561, 1.8731735999537031] 0.03827833823864045


In [127]:
# Update the strategy with the evaluated individuals
toolbox.update(population)

IndexError: tuple index out of range